In [1]:
from utils.general import file_to_lazy_frame
from utils.vitals import analyze_vitals
import pandas as pd
import polars as pl
import numpy as np
import pandas as pd
from scipy import signal
from utils.general import bandpass_filt, shannon_energy

In [3]:
print(pd.__version__)

2.0.0


In [2]:
path = '../ma-data-kit-main/data/test005-CarleSleep008-interp_imu_xl-2023-04-30T02:00:00-05:00-2023-04-30T10:00:00-05:00.parquet'

df = pd.read_parquet(path)[['t', 'x', 'y', 'z']]
df.set_index('t', inplace=True)

df_down = df.resample('1ms').mean().dropna()
fs = 1e-09**-1/np.median(np.diff(df_down.index.values.astype(np.int64)))


# Calculating HR from MA Z signals
ma_z_lp = bandpass_filt(df_down['z'].to_frame(), 1/0.05, fs, 'lowpass')
ma_z_bp = bandpass_filt(ma_z_lp, (20, 51), fs, 'bandpass')

ma_z_bp_cwt = signal.cwt(ma_z_bp['z'].values.T,signal.morlet,np.arange(1,20))
ma_z_cwt_pk = ma_z_bp_cwt.T[:,1:15]


In [3]:
cwt_se = shannon_energy(ma_z_cwt_pk)
cwt_se_gs = ndimage.gaussian_filter(np.abs(cwt_se),sigma=25)


# low-pass to extract waves that have roughly 700 ms width
single_peak = np_bandpass_filt(cwt_se_gs, 1/0.7, fs, 'lowpass')


NameError: name 'ndimage' is not defined

In [3]:
df = analyze_vitals(path)

In [4]:
df

,x,y,z
t,,,
2023-04-30 08:28:59.295,65.0,-3776.0,7197.0
2023-04-30 08:28:59.296,65.0,-3776.0,7209.5
2023-04-30 08:28:59.297,65.0,-3777.0,7167.0
2023-04-30 08:28:59.298,66.0,-3777.0,7202.0
2023-04-30 08:28:59.299,66.5,-3778.0,7188.0
...,...,...,...
2023-04-30 14:59:59.917,7862.0,-1886.0,1403.0
2023-04-30 14:59:59.918,7862.0,-1886.0,1357.0
2023-04-30 14:59:59.919,7862.0,-1886.0,1383.0
